In [1]:
import os
import json

# 目标目录
directory_path = "/mnt/lingjiejiang/multimodal_code/data/dpo/Qwen2-VL-72B-Instruct_generate/"

# 获取目录下的所有 JSON 文件
json_files = [f for f in os.listdir(directory_path) if f.endswith(".json")]

# 读取并合并所有 JSON 文件
merged_data = []

for json_file in json_files:
    file_path = os.path.join(directory_path, json_file)
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            if isinstance(data, list):
                merged_data.extend(data)
            else:
                merged_data.append(data)
    except Exception as e:
        print(f"Error reading {json_file}: {e}")

# 保存合并后的 JSON 文件
# merged_file_path = os.path.join(directory_path, "merged_data.json")
# with open(merged_file_path, "w", encoding="utf-8") as f:
#     json.dump(merged_data, f, ensure_ascii=False, indent=4)

# merged_file_path


In [2]:
len(merged_data)

150476

In [3]:
merged_data[0]

{'id': '5669_47',
 'messages': [{'content': '<image>\nLay out HTML code to reproduce the same design as the website shown in the screenshot. Showcase code created in harmony with material design aesthetics.',
   'role': 'user'},
  {'content': '```html\n<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>Travel Agency</title>\n    <style>\n        body {\n            font-family: Arial, sans-serif;\n            margin: 0;\n            padding: 0;\n            display: flex;\n            flex-direction: column;\n            align-items: center;\n            justify-content: center;\n            background-color: #f4f4f4;\n        }\n\n        .hero {\n            width: 100%;\n            max-width: 1200px;\n            height: 300px;\n            background-color: #d3d3d3;\n            margin-bottom: 20px;\n        }\n\n        .section {\n            text-align: center;\n          

In [11]:
html_data = [item for item in merged_data if item["type"] == "html"]
len(html_data)

100000

In [12]:
chart_data = [item for item in merged_data if item["type"] == "chart"]
len(chart_data)

50476

In [13]:
def filter_data(data):
    new_data = []
    for d in data:
        if d["stop_reason"] == "stop" and d["generate_raw"] != "" and d["code_extract"] != "":
            new_data.append(d)
    return new_data
notempty_chart_data = filter_data(chart_data)
len(notempty_chart_data)

50382

In [16]:
with open("/mnt/lingjiejiang/multimodal_code/data/dpo/Qwen2-VL-72B-Instruct_generate/merged_data/qwen2vl_72b_chart.json", "w", encoding="utf-8") as f:
    json.dump(notempty_chart_data, f, ensure_ascii=False, indent=0)

In [22]:
with open("/mnt/lingjiejiang/multimodal_code/data/dpo/Qwen2-VL-72B-Instruct_generate/merged_data/qwen2vl_72b_chart.json", "r", encoding="utf-8") as f:
    chart_qwen = json.load(f)
len(chart_qwen)
chart_qwen[0]

{'index': 313,
 'evol_text': "import matplotlib.pyplot as plt\nimport numpy as np\n\n# Generate data\nx = np.linspace(0, 10, 200)\ny1 = np.sin(x)\ny2 = np.cos(x)\n\n# Create a new figure and axes\nfig, ax = plt.subplots(figsize=(10, 6))\n\n# Plot the sine and cosine waves with enhanced styles\nax.plot(x, y1, label='Sine Wave', color='C0', linestyle='-', linewidth=2.5, marker='o', markersize=5, markevery=10)\nax.plot(x, y2, label='Cosine Wave', color='C1', linestyle='--', linewidth=2.5, marker='s', markersize=5, markevery=10)\n\n# Set labels with more descriptive titles\nax.set_xlabel('Angle [radians]', fontsize=12, fontweight='bold')\nax.set_ylabel('Amplitude', fontsize=12, fontweight='bold')\nax.set_title('Trigonometric Functions: Sine and Cosine Waves', fontsize=14, fontweight='bold', color='darkblue', pad=20)\n\n# Improve the legend\nax.legend(loc='upper right', fontsize=10, frameon=True, shadow=True, fancybox=True, borderpad=1)\n\n# Add grid for better readability\nax.grid(True, wh

In [17]:
with open("/mnt/lingjiejiang/multimodal_code/data/dpo/chart/github_gpt4o_chart_47k_code.json", "r", encoding="utf-8") as f:
    chart_4o = json.load(f)
len(chart_4o)

47996

In [26]:
import json

chart_4o_path = "/mnt/lingjiejiang/multimodal_code/data/dpo/chart/github_gpt4o_chart_47k_code.json"
chart_qwen_path = "/mnt/lingjiejiang/multimodal_code/data/dpo/Qwen2-VL-72B-Instruct_generate/merged_data/qwen2vl_72b_chart.json"

with open(chart_4o_path, "r", encoding="utf-8") as f:
    chart_4o = json.load(f)

with open(chart_qwen_path, "r", encoding="utf-8") as f:
    chart_qwen = json.load(f)

# 构建 revised_text 到 code_extract 的映射
qwen_revised_map = {item['revised_text']: item['code_extract'] for item in chart_qwen}

# 重新构造消息格式
formatted_data = []
for item in chart_4o:
    revised_text = item["revised_text"]
    chosen_code = item["code_extract"]
    rejected_code = qwen_revised_map.get(revised_text, None)

    if rejected_code:
        formatted_data.append({
            "conversations": [
                {
                    "from": "human",
                    "value": f"<image>\n{item['gpt_prompt']}"
                }
            ],
            "chosen": {
                "from": "gpt",
                "value": chosen_code
            },
            "rejected": {
                "from": "gpt",
                "value": rejected_code
            },
            "images": item["images"]
        })
# # 保存修改后的数据
# output_path = "modified_chart_4o.json"
# with open(output_path, "w", encoding="utf-8") as f:
#     json.dump(chart_4o, f, ensure_ascii=False, indent=4)

# print(f"修改后的数据已保存至 {output_path}")


In [29]:
with open("/mnt/lingjiejiang/multimodal_code/data/dpo/chart/chart_4o_qwen72b_47k.json", "w", encoding="utf-8") as f:
    json.dump(formatted_data, f, ensure_ascii=False, indent=0)

In [25]:
len(chart_4o)

47996

In [18]:
chart_qwen = notempty_chart_data
len(chart_qwen)

50382

In [21]:
chart_4o[:1]

[{'index': 0,
  'code_extract': "import matplotlib.pyplot as plt\n\n# Data\nquarters = ['Q1', 'Q2', 'Q3', 'Q4']\nrevenue_2022 = [150, 200, 250, 300]\nrevenue_2023 = [180, 230, 290, 350]\n\n# Create the plot\nplt.figure(figsize=(10, 6))\nplt.plot(quarters, revenue_2022, 'b-o', label='2022 Revenue')\nplt.plot(quarters, revenue_2023, 'g--s', label='2023 Revenue')\n\n# Highlight the highest revenue point\nplt.annotate('Highest Revenue', xy=('Q4', 350), xytext=('Q4', 360),\n             arrowprops=dict(facecolor='black', shrink=0.05))\n\n# Labels and title\nplt.xlabel('Quarter')\nplt.ylabel('Revenue (in millions)')\nplt.title('Quarterly Revenue Comparison: 2022 vs 2023', fontweight='bold')\n\n# Legend\nplt.legend()\n\n# Grid\nplt.grid(True)\n\n# Show plot\nplt.show()",
  'generate_raw': "To reproduce the graph using Matplotlib with the specified dimensions, you can use the following code:\n\n```python\nimport matplotlib.pyplot as plt\n\n# Data\nquarters = ['Q1', 'Q2', 'Q3', 'Q4']\nrevenue_2

In [20]:
chart_qwen[:1]

[{'index': 313,
  'evol_text': "import matplotlib.pyplot as plt\nimport numpy as np\n\n# Generate data\nx = np.linspace(0, 10, 200)\ny1 = np.sin(x)\ny2 = np.cos(x)\n\n# Create a new figure and axes\nfig, ax = plt.subplots(figsize=(10, 6))\n\n# Plot the sine and cosine waves with enhanced styles\nax.plot(x, y1, label='Sine Wave', color='C0', linestyle='-', linewidth=2.5, marker='o', markersize=5, markevery=10)\nax.plot(x, y2, label='Cosine Wave', color='C1', linestyle='--', linewidth=2.5, marker='s', markersize=5, markevery=10)\n\n# Set labels with more descriptive titles\nax.set_xlabel('Angle [radians]', fontsize=12, fontweight='bold')\nax.set_ylabel('Amplitude', fontsize=12, fontweight='bold')\nax.set_title('Trigonometric Functions: Sine and Cosine Waves', fontsize=14, fontweight='bold', color='darkblue', pad=20)\n\n# Improve the legend\nax.legend(loc='upper right', fontsize=10, frameon=True, shadow=True, fancybox=True, borderpad=1)\n\n# Add grid for better readability\nax.grid(True, 

In [15]:
notempty_chart_data[:1]

[{'index': 313,
  'evol_text': "import matplotlib.pyplot as plt\nimport numpy as np\n\n# Generate data\nx = np.linspace(0, 10, 200)\ny1 = np.sin(x)\ny2 = np.cos(x)\n\n# Create a new figure and axes\nfig, ax = plt.subplots(figsize=(10, 6))\n\n# Plot the sine and cosine waves with enhanced styles\nax.plot(x, y1, label='Sine Wave', color='C0', linestyle='-', linewidth=2.5, marker='o', markersize=5, markevery=10)\nax.plot(x, y2, label='Cosine Wave', color='C1', linestyle='--', linewidth=2.5, marker='s', markersize=5, markevery=10)\n\n# Set labels with more descriptive titles\nax.set_xlabel('Angle [radians]', fontsize=12, fontweight='bold')\nax.set_ylabel('Amplitude', fontsize=12, fontweight='bold')\nax.set_title('Trigonometric Functions: Sine and Cosine Waves', fontsize=14, fontweight='bold', color='darkblue', pad=20)\n\n# Improve the legend\nax.legend(loc='upper right', fontsize=10, frameon=True, shadow=True, fancybox=True, borderpad=1)\n\n# Add grid for better readability\nax.grid(True, 

In [8]:
notempty_html_data[:1]

[{'id': '5346_4',
  'messages': [{'content': '<image>\nLook at the webpage depiction and provide an HTML transcription. Share the code that upholds the standards of material design.',
    'role': 'user'},
   {'content': '```html\n<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>Restaurant Page</title>\n    <style>\n        body {\n            margin: 0;\n            font-family: Arial, sans-serif;\n            background-color: #f9f9f9;\n        }\n        header {\n            background-color: #f1f1f1;\n            padding: 10px;\n            display: flex;\n            justify-content: space-between;\n            align-items: center;\n        }\n        header input[type="text"] {\n            padding: 5px;\n            border: 1px solid #ccc;\n            border-radius: 3px;\n        }\n        header input[type="submit"] {\n            padding: 5px 10px;\n            border

In [7]:
non_empty_data = [d for d in merged_data if d["code_extract"] != ""]
len(non_empty_data)

111411

In [5]:
merged_data[:2]

[{'id': '5669_47',
  'messages': [{'content': '<image>\nLay out HTML code to reproduce the same design as the website shown in the screenshot. Showcase code created in harmony with material design aesthetics.',
    'role': 'user'},
   {'content': '```html\n<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <title>Travel Agency</title>\n    <style>\n        body {\n            font-family: Arial, sans-serif;\n            margin: 0;\n            padding: 0;\n            display: flex;\n            flex-direction: column;\n            align-items: center;\n            justify-content: center;\n            background-color: #f4f4f4;\n        }\n\n        .hero {\n            width: 100%;\n            max-width: 1200px;\n            height: 300px;\n            background-color: #d3d3d3;\n            margin-bottom: 20px;\n        }\n\n        .section {\n            text-align: center;\n      